In [1]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
import pandas as pd


In [2]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None):
        super(StrasbourgSurveyScenario, self).__init__()

        tax_benefit_system = CountryTaxBenefitSystem()
        tax_benefit_system.load_extension('openfisca_france_local')

        input_data_frame = data.get('input_data_frame')
        self.used_as_input_variables = list(
            set(tax_benefit_system.variables.keys()).intersection(
                set(input_data_frame.columns)
                ))

        self.set_tax_benefit_systems(tax_benefit_system)
        self.year = "2021-03"
        self.init_from_data(data = data)


In [3]:
qf = [410, 510, 620, 720, 820, 920, 1030, 1540, 2050, 2051, 2052]

In [32]:
c =pd.Categorical([string.ascii_uppercase[v] for v in range(len(qf))])

In [33]:
c.astype('int')

ValueError: Cannot cast object dtype to int

In [4]:
counts = [4025, 2113, 1832, 1603, 962, 638, 566, 1990, 1157, 2090, 45]

In [29]:
input_data_frame = pd.DataFrame({
    'strasbourg_metropole_quotient_familial': qf,
    'famille_role_index': 0,
    'foyer_fiscal_role_index': 0,
    'menage_role_index': 0,
    'famille_id': pd.Categorical([string.ascii_uppercase[v] for v in range(len(qf))]),
    'foyer_fiscal_id': range(len(qf)),
    'menage_id': range(len(qf)),
    'nombre_individus_inscrits': counts,
    'nombre_familles': counts,
    'nombre_foyers_fiscaux': 1,
    'nombre_menages': 1, 
    'strasbourg_metropole_nombre_repas_cantine': [200, 150, 200, 100, 145, 200, 150, 130, 180, 200, 150]
    })
input_data_frame

,strasbourg_metropole_quotient_familial,famille_role_index,foyer_fiscal_role_index,menage_role_index,famille_id,foyer_fiscal_id,menage_id,nombre_individus_inscrits,nombre_familles,nombre_foyers_fiscaux,nombre_menages,strasbourg_metropole_nombre_repas_cantine
0,410,0,0,0,A,0,0,4025,4025,1,1,200
1,510,0,0,0,B,1,1,2113,2113,1,1,150
2,620,0,0,0,C,2,2,1832,1832,1,1,200
3,720,0,0,0,D,3,3,1603,1603,1,1,100
4,820,0,0,0,E,4,4,962,962,1,1,145
5,920,0,0,0,F,5,5,638,638,1,1,200
6,1030,0,0,0,G,6,6,566,566,1,1,150
7,1540,0,0,0,H,7,7,1990,1990,1,1,130
8,2050,0,0,0,I,8,8,1157,1157,1,1,180
9,2051,0,0,0,J,9,9,2090,2090,1,1,200


In [30]:
data = dict(input_data_frame = input_data_frame)
scenario = StrasbourgSurveyScenario(data = data)

ValueError: Cannot cast object dtype to int64

# Tarif des cantines par familles

In [7]:
tarif_cantine = scenario.simulation.calculate('strasbourg_metropole_tarification_cantine', period = '2021-03')
tarif_cantine

array([1.5 , 2.15, 2.7 , 3.25, 3.8 , 4.4 , 5.1 , 5.8 , 6.3 , 6.9 , 6.9 ],
      dtype=float32)

# Recettes par tranches / types de familles

In [8]:
sum_tranches = counts * tarif_cantine * 200 # Pour 200 repas par an
sum_tranches

array([1207500.        ,  908590.04030228,  989280.01747131,
       1041950.        ,  731119.99082565,  561440.01216888,
        577319.98920441, 2308400.07591248, 1457820.04413605,
       2884200.03986359,   62100.00085831])

# Recettes totales

In [9]:
sum(sum_tranches)

12729720.21074295

In [10]:
scenario.compute_aggregate('strasbourg_metropole_cout_cantine', period = '2021-03', weighted= False)

7986.5

In [11]:
9760.0 / 200

48.8

In [12]:
scenario.weight_variable_by_entity = {'individu': 'nombre_individus_inscrits', 'famille': 'nombre_familles',
                                     'foyer_fiscal': 'nombre_foyers_fiscaux', 'menage': 'nombre_menages'}


In [13]:
scenario.compute_aggregate('strasbourg_metropole_cout_cantine', period = '2021-03')

10666962.5

In [14]:
counts 

[4025, 2113, 1832, 1603, 962, 638, 566, 1990, 1157, 2090, 45]

In [15]:
sum(counts)

17021

In [17]:
import string
string.ascii_uppercase[0]

'A'

In [18]:
[string.ascii_uppercase[v] for v in range(10)]

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']